In [16]:
import requests
import pandas as pd
import time
import numpy as np
from helper import get_spotify_songs_metadata, get_playlist_urls, get_authorization_url, get_access_token

In [2]:
auth_url = get_authorization_url()
print(auth_url)

https://accounts.spotify.com/authorize?client_id=962aeb5559434e8a8ddb2b8c78e6c3d2&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A3000&scope=user-read-private+user-read-email


In [3]:
cookie = "AQA_EkWesltfe4dwi0BePaK6nF6brykBosrPAE9Xuej_Yn2SWhlKyjMdsjWb4rucvZg_Te6mnlaE-6wwVqg7nEz40LcZANYFO0xAtcAvg9KDiPrVX_1Eo6te3yfGYeMV1r0KrDx4wWU9KJ6WATuDkVgtZSQMiZyn7zv_3ncxHUhPA0Ar44YuhDfDMdSptRjZ7De5vNqZBOe_GGjwLw"

In [4]:
access_token = get_access_token(cookie)
print(access_token)

BQAylw3x0gLwWkAUxi_A9pJLVy1Q-_rgSkmWcsfvOL5mwhNmixzqvIcsl8Q5TTASLoRtDvSXiMd5IrqsmZdEOFOXI9PPaQMkRvyyH-hvGtOIY8ubh5SaJ7MJu1OlxY1f0_iY5kBkG84wEBzRBa5GdGNwMNet8D7OKxXP5h153R7uxRD9WIRW6bdCuavIGDnX47KeNB0mVnsy8Q6Z1XuEjA


In [5]:
url = "https://api.spotify.com/v1/me/playlists"

headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    playlists = response.json()['items']
else:
    print(f'Error: {response.status_code} - {response.json()}')

In [9]:
favorite_songs = []

for playlist in playlists:
  song_url = f"https://api.spotify.com/v1/playlists/{playlist['id']}/tracks"
  headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
  }
  response = requests.get(song_url, headers=headers)

  if response.status_code == 200:
    favorite_songs.extend([song['track']['id'] for song in response.json()['items']])

In [13]:
liked_songs = get_spotify_songs_metadata(access_token, favorite_songs, 1, 1)
print(liked_songs.head())
liked_songs.to_parquet('./datasets/cleaned/music-recommendation-system/liked/songs.parquet')

                       id                      title       artist(s)  \
0  1lb3H41lbvAN28N7yqRNL9                    Idea 10  Gibran Alcocer   
1  3kJACFlgn1rg0I0l4XmzLI                     Idea 1  Gibran Alcocer   
2  0yfMign5fsLtw5I4pK73ge                      Solas  Gibran Alcocer   
3  1FCsK0oKWCtDOYRPYSrYgO                    Idea 22  Gibran Alcocer   
4  0Wzv6Y5yj6vVMeN2x2BBdg  Idea 10 (Slowed & Reverb)  Gibran Alcocer   

   popularity  danceability  energy  key  loudness  mode  speechiness  ...  \
0          71         0.339  0.2420    7   -20.446     0       0.0383  ...   
1          65         0.265  0.1460    8   -21.134     0       0.0323  ...   
2          64         0.187  0.3240    4   -17.517     0       0.0331  ...   
3          63         0.328  0.0704    0   -20.147     0       0.0319  ...   
4          62         0.303  0.1620    3   -25.812     0       0.0309  ...   

   valence    tempo            type                                   uri  \
0    0.711   87.053  

In [17]:
genres = ['edm', 'lofi', 'rap', 'raggaeton', 'instrumental', 'jazz', 'classical', 'r&b']

playlists_based_on_genres = get_playlist_urls(access_token, genres)
print(playlists_based_on_genres[:2])

Error: 404 - {'error': {'status': 404, 'message': "Specified id doesn't exist"}}


UnboundLocalError: local variable 'playlists' referenced before assignment

In [47]:
# iterate through a dict called playlists_based_on_genres
unheard_song_ids = []
for key, value in playlists_based_on_genres.items():
    for url in value:
        playlist_url = f"{url}/tracks"
        headers = {
            'Authorization': f'Bearer {access_token}',
            'Content-Type': 'application/json'
        }
        response = requests.get(playlist_url, headers=headers)
        if response.status_code == 200:
            songs = response.json()['items']
            time.sleep(0.5)
        else:
            print(f'Error: {response.status_code} - {response.json()}')

        for song in songs:
            if song['track']['id'] not in unheard_song_ids:
                unheard_song_ids.append(song['track']['id'])

In [49]:
len(unheard_song_ids)

2298

In [60]:
unheard_songs = pd.DataFrame()

for idx, song_id in enumerate(unheard_song_ids):
  if idx % 100 == 0:
    time.sleep(10)

  headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
  }

  track_response = requests.get(
    f"https://api.spotify.com/v1/tracks/{song_id}",
    headers=headers
  ).json()
  
  audio_analysis_response = requests.get(
    f"https://api.spotify.com/v1/audio-features/{song_id}", 
    headers=headers
  ).json()

  time.sleep(0.1)
  row = {
    'id': track_response.get('id'),
    'title': track_response.get('name'),
    'artist(s)': ', '.join([artist['name'] for artist in track_response.get('album').get('artists')]),
    'popularity': track_response.get('popularity'),
    'danceability': audio_analysis_response.get('danceability'),
    'energy': audio_analysis_response.get('energy'),
    'key': audio_analysis_response.get('key'),
    'loudness': audio_analysis_response.get('loudness'),
    'mode': audio_analysis_response.get('mode'),
    'speechiness': audio_analysis_response.get('speechiness'),
    'acousticness': audio_analysis_response.get('acousticness'),
    'instrumentalness': audio_analysis_response.get('instrumentalness'),
    'liveness': audio_analysis_response.get('liveness'),
    'valence': audio_analysis_response.get('valence'),
    'tempo': audio_analysis_response.get('tempo'),
    'type': audio_analysis_response.get('type'),
    'id': audio_analysis_response.get('id'),
    'uri': audio_analysis_response.get('uri'),
    'track_href': audio_analysis_response.get('track_href'),
    'analysis_url': audio_analysis_response.get('analysis_url'),
    'duration_ms': audio_analysis_response.get('duration_ms'),
    'time_signature': audio_analysis_response.get('time_signature'),
    'listen': 0,
    'like': np.nan
  }
  unheard_songs = pd.concat([unheard_songs, pd.DataFrame(row, index=[len(unheard_songs)])])

In [67]:
unheard_songs.to_parquet('./datasets/cleaned/music-recommendation-system/unheard/songs.parquet', engine='pyarrow')

In [72]:
discover_weekly_song_ids = []
discover_weekly_url = "https://api.spotify.com/v1/playlists/37i9dQZEVXcHBp4tw3uD8b/tracks"

headers = {
  'Authorization': f'Bearer {access_token}',
  'Content-Type': 'application/json'
}

response = requests.get(discover_weekly_url, headers=headers)

if response.status_code == 200:
  discover_weekly_song_ids = [song['track']['id'] for song in response.json()['items']]
  time.sleep(0.5)
else:
  print(f'Error: {response.status_code} - {response.json()}')

print(discover_weekly_song_ids)

['1s7oOCT8vauUh01PbJD6ps', '1elhmWW7Bv0MOQj2gAsyoV', '1OyakDSOcvuZjvESJxri3w', '3TFdXrYIubXABQh0CxIiSs', '6TkyRlfE4XMPDR6uA0xYyz', '5cRalo5aaLDFcNudQEQqJx', '3kyVmdoNCvLxGlmqY1yTQz', '69g0XAms2Js45eTCrT1nVL', '16C6yAxcA3GCtoXbq3dUYl', '1lF7uD9Yu2CL17a6tm4Tkh', '67u4rpzdgdiPjRGZii9GEz', '73tnLQtl7FqD0MS7VEM0xi', '3nzK0WPzGPczWa45eIFphc', '1OBK4DltPHWoeilauCyckU', '6YULY6STClTO2tekAZb81j', '5tgLkUlAmTR9FhOzVVfIJS', '7B5EbapjMpLuK7A7PJUvxn', '6UdU7dLYydmt6OaPItTso9', '6tm4RYbhLGrFmYwSTFNFaB', '7s0pdQmbXq6GtMHpkCTLLw', '23xnJ8uEgUPXLARnsedrOn', '2VbO6uAFOFVaAuWGbntoQL', '6vhrwbHQjl4klPXlPLnar4', '4UoeRYvYxVx72vWK4lGRYv', '3XFFxrFN9s8hoIO5TwKxIT', '2F8Sx9EPW0bcqTMZDwxEqX', '3cTsh9sZdHHV6Syjcwg8VW', '4Gzpzsew2wogr4iQSlT54W', '3XCoEJvCYqTbM8bhples8N', '3vN4noEugIxWThJUeEJqQJ']


In [73]:
discover_weekly_songs = pd.DataFrame()

for song_id in discover_weekly_song_ids:
  headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
  }

  track_response = requests.get(
    f"https://api.spotify.com/v1/tracks/{song_id}",
    headers=headers
  ).json()
  
  audio_analysis_response = requests.get(
    f"https://api.spotify.com/v1/audio-features/{song_id}", 
    headers=headers
  ).json()

  time.sleep(0.1)
  row = {
    'id': track_response.get('id'),
    'title': track_response.get('name'),
    'artist(s)': ', '.join([artist['name'] for artist in track_response.get('album').get('artists')]),
    'popularity': track_response.get('popularity'),
    'danceability': audio_analysis_response.get('danceability'),
    'energy': audio_analysis_response.get('energy'),
    'key': audio_analysis_response.get('key'),
    'loudness': audio_analysis_response.get('loudness'),
    'mode': audio_analysis_response.get('mode'),
    'speechiness': audio_analysis_response.get('speechiness'),
    'acousticness': audio_analysis_response.get('acousticness'),
    'instrumentalness': audio_analysis_response.get('instrumentalness'),
    'liveness': audio_analysis_response.get('liveness'),
    'valence': audio_analysis_response.get('valence'),
    'tempo': audio_analysis_response.get('tempo'),
    'type': audio_analysis_response.get('type'),
    'id': audio_analysis_response.get('id'),
    'uri': audio_analysis_response.get('uri'),
    'track_href': audio_analysis_response.get('track_href'),
    'analysis_url': audio_analysis_response.get('analysis_url'),
    'duration_ms': audio_analysis_response.get('duration_ms'),
    'time_signature': audio_analysis_response.get('time_signature'),
    'listen': 0,
    'like': np.nan
  }
  discover_weekly_songs = pd.concat([discover_weekly_songs, pd.DataFrame(row, index=[len(discover_weekly_songs)])])

print(len(discover_weekly_songs))

30


In [74]:
discover_weekly_heard_list = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
discover_weekly_like_list = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
discover_weekly_songs['listen'] = discover_weekly_heard_list
discover_weekly_songs['like'] = discover_weekly_like_list

In [76]:
discover_weekly_songs.to_parquet('./datasets/cleaned/music-recommendation-system/unheard/discover-weekly.parquet', engine='pyarrow')